In [1]:
import os
import glob
import numpy as np
import librosa
import soundfile
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier

# Defining all emotions
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# Include all emotions in the classification
AVAILABLE_EMOTIONS = set(int2emotion.values())  # Includes all 8 emotions

# Feature extraction function
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    try:
        with soundfile.SoundFile(file_name) as sound_file:
            X = sound_file.read(dtype="float32")
            sample_rate = sound_file.samplerate
            stft = np.abs(librosa.stft(X)) if chroma else None

            result = np.array([])
            if mfcc:
                mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
                result = np.hstack((result, mfccs))
            if chroma:
                chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
                result = np.hstack((result, chroma))
            if mel:
                mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
                result = np.hstack((result, mel))
        return result
    except Exception as e:
        print(f"⚠️ Error processing {file_name}: {e}")
        return None

#  Loading dataset and training model
def load_dataset(data_path, test_size=0.2):
    X, y = [], []

    files = glob.glob(os.path.join(data_path, "*.wav"))
    print(f"🔍 Found {len(files)} audio files.")

    for file in files:
        basename = os.path.basename(file)
        parts = basename.split("-")

        if len(parts) < 3:
            print(f"⚠️ Skipping {basename} (Invalid filename format)")
            continue

        emotion_id = parts[2]  # Extract emotion ID
        emotion = int2emotion.get(emotion_id)

        if emotion is None:
            print(f"⚠️ Skipping {basename} (Unknown emotion ID: {emotion_id})")
            continue

        features = extract_feature(file)
        if features is not None:
            X.append(features)
            y.append(emotion)

    if not X:
        raise ValueError("❌ No valid audio features extracted!")

    X, y = np.array(X), np.array(y)
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return train_test_split(X, y, test_size=test_size, random_state=42), encoder, scaler

# Training the model
def train_model(data_path="/content/Test"):
    (X_train, X_test, y_train, y_test), encoder, scaler = load_dataset(data_path)

    model = MLPClassifier(
        alpha=0.01, batch_size=256, epsilon=1e-08,
        hidden_layer_sizes=(300,), learning_rate="adaptive", max_iter=500
    )

    print("🚀 Training model...")
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test) * 100
    print(f"✅ Model trained with accuracy: {accuracy:.2f}%")

    # Save model & encoder
    joblib.dump(model, "emotion_model.pkl")
    joblib.dump(encoder, "encoder.pkl")
    joblib.dump(scaler, "scaler.pkl")
    return model, encoder, scaler

# Predicting emotions from new audio files
def predict_emotions(audio_folder="/Test"):
    # Load trained model
    model = joblib.load("emotion_model.pkl")
    encoder = joblib.load("encoder.pkl")
    scaler = joblib.load("scaler.pkl")

    files = glob.glob(os.path.join(audio_folder, "*.wav"))
    print(f"🔎 Detecting emotions in {len(files)} files...")

    for file in files:
        features = extract_feature(file)
        if features is None:
            print(f"⚠️ Skipping {os.path.basename(file)} (Feature extraction failed)")
            continue

        features = scaler.transform([features])  # Normalize input
        emotion_index = model.predict(features)[0]  # Get prediction
        emotion = encoder.inverse_transform([emotion_index])[0]

        print(f"🎵 {os.path.basename(file)} → Detected Emotion: {emotion}")

# Training model and predict
train_model("/content/Test")
predict_emotions("/content/Test")


🔍 Found 0 audio files.


ValueError: ❌ No valid audio features extracted!